In [0]:


import datetime
import psycopg2
from contextlib import closing
dbinfo = {
    'NAME': 'dailyest',
    'USER': 'app_bdp_usage_qa',
    'PASSWORD': '2mHdFW6%#REu',
    'HOST': 'internal-aa-prod-plproxy-internal-4-329644124.us-east-1.elb.amazonaws.com',
    'PORT': 7432,
}
conn = psycopg2.connect("""
            dbname={dbname} user={user} host={host} port={port} password={passwd}
            """.format(dbname=dbinfo['NAME'], user=dbinfo['USER'], passwd=dbinfo['PASSWORD'],
                       host=dbinfo['HOST'], port=dbinfo['PORT']))
def select(stmt, params):
    with closing(conn.cursor()) as cur:
        cur.execute(stmt, params)
        rows = cur.fetchall()
        cnt = cur.rowcount
    return rows, cnt


sql = """select device_id, date, store_id, kpi, category_id, app_type_id, purchase_type_id, estimate from plproxy.execute_select_nestloop($proxy$ select device_id, date, store_id, kpi, category_id, app_type_id, purchase_type_id, estimate from ms.market_size_daily_estimate_1000 where date = '2020-01-10' and purchase_type_id=12 and app_type_id=2 and store_id=14 and category_id in (65) $proxy$) tbl (device_id SMALLINT, date DATE, store_id INT, kpi SMALLINT, category_id INT, app_type_id SMALLINT, purchase_type_id SMALLINT, estimate BIGINT);"""
rows, _ = select(sql, None)
print len(rows)

for row in rows:
    print row




In [0]:
%%sh

PGPASSWORD='2mHdFW6%#REu' psql -h internal-aa-prod-plproxy-internal-4-329644124.us-east-1.elb.amazonaws.com -U app_bdp_usage_qa -d dailyest -p 7432 << EOF 
select date, sum(count_a) from plproxy.execute_select_nestloop(\$proxy\$ 
    select date, count(pub_id) as counta
    from pp.pub_store_daily_estimate_0
    where 
        DATE between '2010-07-04' and '2010-07-04'
    group by date
\$proxy\$) tbl (date DATE, count_a BIGINT) group by date order by date asc;

--\$proxy\$) tbl (device_id SMALLINT, date DATE, store_id INT, kpi SMALLINT, category_id INT, app_type_id SMALLINT, purchase_type_id SMALLINT, estimate BIGINT);
--2010-07-04, 10575875, 10575875, 0
--------------+----------
-- 2010-07-04 | 10575875

EOF



In [0]:
%%sh

time PGPASSWORD='wZw8cfBuuklIskVG' psql -h 10.2.6.141  -U citus_bdp_prod_app_int_qa -d aa_store_db -p 5432 << EOF 
EXPLAIN ANALYZE
SELECT sum(est_revenue) FROM store.store_est_publisher_fact_v2 WHERE date BETWEEN '2017-05-01' AND '2017-05-31';
EOF

time PGPASSWORD='wZw8cfBuuklIskVG' psql -h 10.2.6.141  -U citus_bdp_prod_app_int_qa -d aa_store_db -p 5432 << EOF 
EXPLAIN ANALYZE
SELECT sum(est_revenue) FROM store.store_est_publisher_fact_v2 WHERE granularity='daily' AND date in ('2018-01-01', '2018-01-31');
EOF



#category_id=400070
# device_code="android-all"
# country_code="ZA"
# app_price_type_id=2
# purchase_type_id=12


In [0]:
%%sh

time PGPASSWORD='wZw8cfBuuklIskVG' psql -h 10.2.6.141  -U citus_bdp_prod_app_int_qa -d aa_store_db -p 5432 << EOF 
\x
select * from pg_indexes where tablename='store_est_publisher_fact_v2';
EOF




In [0]:

from pyspark.sql.functions import lit, udf
from pyspark.sql.types import (
    ArrayType, BooleanType, LongType, IntegerType,
    StringType, StructType, StructField
)
schema = StructType([
    StructField("store_id", IntegerType(), False),
    StructField("date", StringType(), False),
    StructField("platform_id", IntegerType(), False),
    StructField("device", StringType(), False),
    StructField("data_type", StringType(), False),
    StructField("price_type", IntegerType(), False),
    StructField("purchase_type", IntegerType(), False),
    StructField("category_id", IntegerType(), False),
    StructField("estimate", LongType(), False)
])
# sample unified data:
# spark.read.parquet("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.market-size.v1/fact/granularity=daily/date=2020-01-10/").filter("country_code='US' and app_price_type_id=0 and purchase_type_id=10 and device_code='android-all' and category_id=400002").show()
# raw data
print spark.read.csv("s3://b2c-prod-dca-store-estimates/store_est/v_final/DAY/2020-01-10/ios/market_size/*", schema=schema, sep='\t').show(1)
# print spark.read.csv(["s3://b2c-prod-dca-store-estimates/store_est/v_final/DAY/2020-01-10/android/market_size/*", "s3://b2c-prod-dca-store-estimates/store_est/v_final/DAY/2020-01-10/ios/market_size/*"], schema=schema, sep='\t').count()



In [0]:


# spark.read.parquet("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est-publisher.v1/fact/granularity=daily/date=2020-01-01").show(10)
# spark.read.parquet("s3://b2c-prod-dca-store-estimates/store_estv2/PUB_ESTIMATES_FINAL/version=2.0.0/range_type=DAY/date=2020-01-01/").select('store_id').distinct().show(9999)
# spark.read.parquet("s3://b2c-prod-dca-store-estimates/store_estv2/PUB_ESTIMATES_FINAL/version=2.0.0/range_type=DAY/date=2020-01-01/").select('category_id').distinct().show(9999)
# spark.read.parquet("s3://b2c-prod-dca-store-estimates/store_estv2/PUB_ESTIMATES_FINAL/version=2.0.0/range_type=DAY/date=2020-01-01/").select('feed').distinct().show(9999)

spark.read.parquet("s3://b2c-prod-dca-store-estimates/store_estv2/PUB_ESTIMATES_FINAL/version=2.0.0/range_type=DAY/date=2020-01-01/").select('platform').distinct().show(9999)
spark.read.parquet("s3://b2c-prod-dca-store-estimates/store_estv2/PUB_ESTIMATES_FINAL/version=2.0.0/range_type=DAY/date=2020-01-01/").select('platform_id').distinct().show(9999)


# print spark.read.parquet("s3://b2c-prod-dca-store-estimates/store_estv2/PUB_ESTIMATES_FINAL/version=2.0.0/range_type=DAY/date=2020-01-01/").filter("store_id in (10, 143441)").count()
# df = spark.read.parquet("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est-publisher.v1/fact/granularity=daily/date=2020-01-01").filter("country_code = 'US'")
# metrics_count = 0
# for metric in ["free_app_download", "paid_app_download", "revenue"]:
#     metrics_count += df.filter("{} is not null".format(metric)).count()
# print metrics_count




In [0]:
%%sh

# aws s3 ls s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est-publisher.v1/fact/granularity=daily/date=2020-01-01/device_code=ios-all/
# aws s3 ls s3://b2c-prod-dca-store-estimates/store_estv2/PUB_ESTIMATES_FINAL/version=2.0.0/range_type=DAY/date=2020-01-01/platform=ios/
# aws s3 ls s3://b2c-prod-dca-store-estimates/store_estv2/PUB_ESTIMATES_FINAL/version=2.0.0/range_type=DAY/date=2020-01-01/platform=android/

aws s3 ls s3://b2c-prod-dca-store-estimates/store_estv2/PUB_ESTIMATES_FINAL/version=2.0.0/range_type=DAY/


In [0]:

spark.sparkContext.addPyFile("/home/hadoop/bdp/application/libs/python/dependencies.zip")
import pandas as pd
pd.set_option('expand_frame_repr', False)




In [0]:


# Copyright (c) 2018 App Annie Inc. All rights reserved.
# pylint: disable=E1101,C0412,C1801

"""
DB Check modules
"""

import datetime
from applications.db_check_v1.common.base_test import PipelineTest
from applications.db_check_v1.common.db_check_utils import query_df, etl_skip
from applications.db_check_v1.common.constants import COUNTRY_CODE_MAPPING_BY_MARKET_CODE as COUNTRY_CODE_MAPPING, \
    CATEGORY_ID_MAPPING_BY_MARLKET_AND_DEVICE_CODE as CATEGORY_ID_MAPPING
from applications.db_check_v1.common.utils import get_week_start_end_date, get_date_list
#from applications.db_check_v1.cases.store.publisher_est_v1.constants import MARKET_SIZE_DSN


class PublisherEstRawData(object):
    raw_s3_path = "s3://b2c-prod-dca-store-estimates/store_estv2/PUB_ESTIMATES_FINAL/version=2.0.0/range_type=DAY" \
                  "/date={}/"
    device_code_mapping = {
        "00": "android-all",
        "01": "android-all",
        "02": "android-all",
        "10": "ios-phone",
        "11": "ios-phone",
        "12": "ios-phone",
        "1100": "ios-tablet",
        "1101": "ios-tablet",
        "1102": "ios-tablet",
        "21000": "ios-all",
        "21001": "ios-all",
        "21002": "ios-all",
    }

    metric_mapping = {
        0: "free_app_download",
        1: "paid_app_download",
        2: "revenue",
        101: "free_app_download",
        100: "paid_app_download",
        102: "revenue",
        1000: "free_app_download",
        1001: "paid_app_download",
        1002: "revenue"
    }

    dimension_mapping = {
        "id": "publisher_id",
    }

    def __init__(self, spark):
        self.spark = spark

    def get(self, date, country_code):
        df = self._get_raw_data_by_date_country(date, country_code)
        df = self._parse_mapping(df)
        df = self._parse_unified_format(df)
        df = self._data_clean_up(df)
        return df

    def _data_clean_up(self, df):
        # clean unknown mapping
        category_id_list = list(set(CATEGORY_ID_MAPPING["apple-store"]["ios-all"].values() +
                                    CATEGORY_ID_MAPPING["google-play"]["android-all"].values()))

        country_code_list = list(set(COUNTRY_CODE_MAPPING["apple-store"].values() +
                                     COUNTRY_CODE_MAPPING["google-play"].values()))

        df = df[(df['category_id'].isin(category_id_list)) & (df['country_code'].isin(country_code_list))]
        return df

    def _parse_mapping(self, df):
        # country_code mapping
        df.loc[df["platform_id"] == 0] = df.loc[df["platform_id"] == 0].\
            replace({"store_id": COUNTRY_CODE_MAPPING["google-play"]})
        df.loc[df["platform_id"] == 1] = df.loc[df["platform_id"] == 1].\
            replace({"store_id": COUNTRY_CODE_MAPPING["apple-store"]})
        df = df.rename(columns={'store_id': 'country_code'})

        # category_id mapping
        df.loc[df["platform_id"] == 0] = df.loc[df["platform_id"] == 0].\
            replace({"category_id": CATEGORY_ID_MAPPING["google-play"]["android-all"]})
        df.loc[df["platform_id"] == 1] = df.loc[df["platform_id"] == 1].\
            replace({"category_id": CATEGORY_ID_MAPPING["apple-store"]["ios-all"]})

        # device_code mapping
        df["device_code"] = df["platform_id"].astype(str) + df["feed"].astype(str)
        df = df.replace({"device_code": self.device_code_mapping})

        # granularity
        df["granularity"] = "daily"

        # metrics mapping (from feed)
        df = df.replace({"feed": self.metric_mapping})
        return df

    def _parse_unified_format(self, df):
        df = df.rename(columns=self.dimension_mapping)
        df = df.pivot_table(index=["publisher_id", "category_id", "device_code", "country_code", "granularity"],
                            columns='feed', values='est')
        df.reset_index(inplace=True)
        df.columns.name = None
        return df

    def _get_raw_data_by_date_country(self, date, country_code):
        """
        +----------+--------+-----------+-----------+--------+----+----+-----+--------+
        |        id|store_id|category_id|platform_id|vertical|rank|feed|  est|platform|
        +----------+--------+-----------+-----------+--------+----+----+-----+--------+
        | 284417353|       0|       6006|          1|       1|   1|1002|45235|     ios|
        | 349554266|       0|       6006|          1|       1|   2|1002|20732|     ios|
        |1316153435|       0|       6006|          1|       1|   3|1002|15136|     ios|
        +----------+--------+-----------+-----------+--------+----+----+-----+--------+
        """
        ios_store_ids = [str(k) for k, v in COUNTRY_CODE_MAPPING["apple-store"].items() if v == country_code]
        gp_store_ids = [str(k) for k, v in COUNTRY_CODE_MAPPING["google-play"].items() if v == country_code]
        raw_df = self.spark.read.parquet(self.raw_s3_path.format(date)).\
            filter('store_id in ({})'.format(",".join(ios_store_ids + gp_store_ids))).toPandas()
        return raw_df

    def get_metrics_count(self, date):
        ios_store_id = [str(s_id) for s_id in COUNTRY_CODE_MAPPING["apple-store"].keys()]
        gp_store_id = [str(s_id) for s_id in COUNTRY_CODE_MAPPING["google-play"].keys()]
        # ios_category_id = [str(c_id) for c_id in CATEGORY_ID_MAPPING["apple-store"]["ios-all"].keys()]
        # gp_category_id = [str(c_id) for c_id in CATEGORY_ID_MAPPING["google-play"]["android-all"].keys()]

        fillter_sql = "platform_id = {} and store_id in ({})"
        df = self.spark.read.parquet(self.raw_s3_path.format(date))
        count_all = df.filter(fillter_sql.format(1, ",".join(ios_store_id))).count() + \
                    df.filter(fillter_sql.format(0, ",".join(gp_store_id))).count()
        return count_all

    def get_v1_raw_metrics_count(self, date):
        ios_store_id = [str(s_id) for s_id in COUNTRY_CODE_MAPPING["apple-store"].keys()]
        gp_store_id = [str(s_id) for s_id in COUNTRY_CODE_MAPPING["google-play"].keys()]

        df = self.spark.read.option("delimiter", "\t").csv(
            "s3://b2c-prod-dca-store-estimates/store_est/v_final/DAY/{}/*/sbe_est_publisher/*/*.csv.gz".format(date))
        fillter_sql = "_c2 = {} and _c0 in ({})"  # _c2 > platform_id, _c0 > store_id

        count_all = df.filter(fillter_sql.format(1, ",".join(ios_store_id))).count() + \
                    df.filter(fillter_sql.format(0, ",".join(gp_store_id))).count()
        return count_all


class PublisherEstUnifiedData(object):
    unified_s3_path = "s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est-publisher.v1/fact/" \
                      "granularity=daily/date={}/"

    def __init__(self, spark):
        self.spark = spark

    def get(self, date, country_code):
        unified_df = self.spark.read.format("delta").\
            load(self.unified_s3_path.format(date)).filter("country_code = '{}'".format(country_code)).toPandas()
        unified_df = unified_df.drop(["_identifier", "revenue_iap", "revenue_non_iap", "date"], axis=1)
        return unified_df

    def get_metrics_count(self, date):
        df = self.spark.read.format("delta").load(self.unified_s3_path.format(date))
        metrics_count = 0
        for metric in ["free_app_download", "paid_app_download", "revenue"]:
            metrics_count += df.filter("{} is not null".format(metric)).count()
        return metrics_count



class PublisherEstDBData(object):
    def get(self, date):
        sql = "SELECT * FROM store.store_publisher_est_fact_v1 WHERE date='{}'".format(date)
        return query_df(MARKET_SIZE_DSN, sql)

    def get_led(self):
        sql = "SELECT * FROM store.store_publisher_est_latest_date_v1"
        return query_df(MARKET_SIZE_DSN, sql)


class TestPublisherEstWeekly(PipelineTest):
    # Every Tuesday 16:00 UTC time will refresh the data of last Full Week.
    trigger_date_config = ('* 16 * * 2', 3)

    def _compare_df(self, df1, df2, log=''):
        for diff_type in ["left", "right"]:
            diff_df = df1.merge(df2, indicator=True, how=diff_type)  # .loc[lambda x : x['_merge']!='both']
            diff_df = diff_df.loc[diff_df["_merge"] != "both"]
            if len(diff_df) != 0:
                print diff_type
                print "dataframe overview of df1 and df2"
                print df1
                print df2
                print "dimension overview of diff df"
                print diff_df.country_code.unique()
                print diff_df.category_id.unique()
                print diff_df.device_code.unique()
            self.assertEqual(len(diff_df), 0,
                             msg="found mismatch when compare the raw, unified, db."
                                 " diff count is \n {}, logs:{}".format(len(diff_df), log))

    @etl_skip()
    def test_publisher_est_etl_accuracy(self):
        # Every Tuesday 16:00 UTC time will refresh the data of last Full Week.
        country_code = 'US'
        start_date, end_date = get_week_start_end_date(self.check_date_str)
        date_list = get_date_list(start_date, end_date)
        for date in date_list:
            raw_df = PublisherEstRawData(self.spark).get(date, country_code)
            unified_df = PublisherEstUnifiedData(self.spark).get(date, country_code)
            # db_df = PublisherEstDBData().get(date)

            self._compare_df(raw_df, unified_df, log="raw / unified - {}".format(date))
            # self._compare_df(unified_df, db_df, log="unified / db - {}".format(date))

    @etl_skip()
    def test_publisher_Est_etl_completeness(self):
        start_date, end_date = get_week_start_end_date(self.check_date_str)
        date_list = get_date_list(start_date, end_date)
        for date in date_list:
            raw_count = PublisherEstRawData(self.spark).get(date)
            unified_count = PublisherEstUnifiedData(self.spark).get(date)
            self.assertEqual(raw_count, unified_count)

    def test_publisher_est_etl_timelines(self):
        # Every Tuesday 16:00 UTC time will refresh the data of last Full Week.
        # E.g. 2020-02-11 17:00 the data of 2020-02-02 ~ 2020-02-08 will be ready
        trigger_datetime = datetime.datetime.strptime("2020-02-11 17:00:00", '%Y-%m-%d %H:%M:%S')
        check_date_str_actual = self._get_check_date_from_routing_config(trigger_datetime).strftime("%Y-%m-%d")
        self.assertEqual("2020-02-08", check_date_str_actual)

In [0]:


from datetime import datetime
def get_date_list(begin_date, end_date, freq):
    date_list = [x.strftime('%Y-%m-%d') for x in list(pd.date_range(start=begin_date, end=end_date, freq=freq))]
    return date_list

begin_date = datetime(2019, 07, 14)
end_date = datetime(2020, 06, 06)

DATE_GRANULARITY_MAPPINGLIST = {
    "daily": get_date_list(begin_date, end_date, "D"),
    "weekly": get_date_list(begin_date, end_date, "W-SAT"),
    "monthly": get_date_list(begin_date, end_date, "M")
}


def debug(case_list):
    """
    method of debug cases in zepplin, result will be print in standard output

    e.g. test case list example
    TestMarketSizeWeekly(trigger_datetime=t_date, methodName='test_market_size_etl_accuracy_and_completeness')
    """
    std_out_origin = sys.stdout
    std_err_origin = sys.stderr
    try:
        suite = unittest.TestSuite()
        for case in case_list:
            suite.addTest(case)
        runner = unittest.TextTestRunner(verbosity=2, buffer=True)
        runner.run(suite)
    except Exception as ex:
        print dir(ex)
        print ex.message
        traceback.print_exception(type(ex), ex, ex.__traceback__)
    finally:
        sys.stdout = std_out_origin
        sys.stderr = std_err_origin


def load_context_zepplin(spark):
    """
    load context in zepplin
    """
    spark.sparkContext.addPyFile("/home/hadoop/bdp/application/libs/python/dependencies.zip")
    spark.sparkContext.addPyFile("/home/hadoop/bdp/application/code.zip")

def _compare_df(self, df1, df2, log=''):
    for diff_type in ["left", "right"]:
        diff_df = df1.merge(df2, indicator=True, how=diff_type)  # .loc[lambda x : x['_merge']!='both']
        diff_df = diff_df.loc[diff_df["_merge"] != "both"]
        if len(diff_df) != 0:
            print diff_type
            print "dataframe overview of df1 and df2"
            print df1
            print df2
            print "dimension overview of diff df"
            print diff_df.country_code.unique()
            print diff_df.category_id.unique()
            print diff_df.device_code.unique()
        self.assertEqual(len(diff_df), 0,
                         msg="found mismatch when compare the raw, unified, db."
                             " diff count is \n {}, logs:{}".format(len(diff_df), log))
        print "PASS - {}".format(log)



In [0]:


begin_date = datetime.datetime(2010, 07, 04)
end_date = datetime.datetime(2019, 07, 13)


date_list = get_date_list(begin_date, end_date, "D")

for date in date_list:
    # count1 = PublisherEstRawData(spark).get_metrics_count(date)
    count1 = PublisherEstRawData(spark).get_v1_raw_metrics_count(date)
    
    count2 = PublisherEstUnifiedData(spark).get_metrics_count(date)
    print "{}, {}, {}, {}".format(date, count1, count2, count1-count2)


In [0]:


raw_df.        
category_id_list = list(set(CATEGORY_ID_MAPPING["apple-store"]["ios-all"].values() +
                                    CATEGORY_ID_MAPPING["google-play"]["android-all"].values()))

        country_code_list = list(set(COUNTRY_CODE_MAPPING["apple-store"].values() +
                                     COUNTRY_CODE_MAPPING["google-play"].values()))

        df = df[(df['category_id'].isin(category_id_list)) & (df['country_code'].isin(country_code_list))]


In [0]:


# legacy_raw_df = MarketSizeRawData(spark)._get_raw_data_by_date("2020-01-01")
# print legacy_raw_df.loc[(legacy_raw_df["category_id"]==36) & (legacy_raw_df["device"]!="google-play")]
print raw_df.loc[(raw_df["category_id"]==100000) & (raw_df["device_code"]!="android-all")]


In [0]:


from applications.db_check_v1.cases.store.app_rank_v1.constants import citus_dsn
from applications.db_check_v1.common.db_check_utils import query_df
from datetime import datetime
date_list = DATE_GRANULARITY_MAPPINGLIST["daily"]
date_list = ["2020-01-09", "2020-01-10", "2020-01-29"]


for date in date_list:
    print '*'*100
    print date
    start=datetime.today()
    # legacy_raw_df = MarketSizeRawData(spark)._get_raw_data_by_date(date)
    raw_df = MarketSizeRawData(spark).get(date)

    # unified_df =  spark.read.parquet("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.market-size.v1/fact/granularity=daily/date={}/".format(date)).toPandas()
    # unified_df = unified_df.drop(["_identifier"], axis=1)
    
    # legacy_unified_df = spark.read.parquet("s3://b2c-prod-data-pipeline-unified-store-paid/unified/app-tech.store.market-size.v1/fact/granularity=daily/date={}/".format(date)).toPandas()
    # legacy_unified_df = legacy_unified_df.drop(["_identifier"], axis=1)
    # legacy_unified_df["category_id"] = pd.to_numeric(legacy_unified_df["category_id"])
    
    sql = "SELECT * FROM store.store_market_size_fact_v1  WHERE date='{}'".format(date)
    db_df = query_df(citus_dsn, sql)

    print datetime.today() - start


    df1= raw_df
    # df2= unified_df
    # df2= legacy_unified_df
    df2= db_df
    for diff_type in ["left", "right"]:
        diff_df = df1.merge(df2,indicator = True, how=diff_type)#.loc[lambda x : x['_merge']!='both']
        diff_df = diff_df.loc[ diff_df["_merge"]!="both"]
        if len(diff_df)!=0:
            print len(df1)
            print len(df2)
            print diff_type
            print len(diff_df)
            print diff_df
            print diff_df.country_code.unique()
            print diff_df.category_id.unique()
            print diff_df.device_code.unique()


In [0]:



# print raw_df.loc[raw_df["est_market_size_revenue"]==15017778.5]
# print unified_df.loc[unified_df["est_market_size_revenue"]==15017778.5]
# print legacy_raw_df

date = "2020-01-01"

# legacy_raw_df = MarketSizeRawData(spark)._get_raw_data_by_date(date)
raw_df = MarketSizeRawData(spark).get(date)

unified_df =  spark.read.parquet("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.market-size.v1/fact/granularity=daily/date={}/".format(date)).toPandas()
# unified_df = unified_df.drop(["_identifier"], axis=1)

# legacy_unified_df = spark.read.parquet("s3://b2c-prod-data-pipeline-unified-store-paid/unified/app-tech.store.market-size.v1/fact/granularity=daily/date={}/".format(date)).toPandas()
# legacy_unified_df = legacy_unified_df.drop(["_identifier"], axis=1)
# legacy_unified_df["category_id"] = pd.to_numeric(legacy_unified_df["category_id"])


#        app_price_type_id  purchase_type_id  category_id  device_code country_code  est_market_size_download  est_market_size_revenue        date granularity  _disable_idx_4_query      _merge
# 183842                  1                11       400007  android-all           CN                      69.0                      NaN  2020-01-09       daily                     0  right_only

category_id=400007
device_code="android-all"
country_code="CN"
app_price_type_id=1
purchase_type_id=12

# print legacy_raw_df.loc[(legacy_raw_df["category_id"]==44) & (legacy_raw_df["device"]=="google-play")  &  (legacy_raw_df["store_id"]==3)  & (legacy_raw_df["price_type"]==app_price_type_id)  & (legacy_raw_df["purchase_type"]==purchase_type_id)]
print legacy_raw_df.loc[(legacy_raw_df["category_id"]==44) & (legacy_raw_df["store_id"]==3)]


print raw_df.loc[(raw_df["category_id"]==category_id) & (raw_df["device_code"]==device_code) & (raw_df["country_code"]==country_code) & (raw_df["app_price_type_id"]==app_price_type_id) & (raw_df["purchase_type_id"]==purchase_type_id) ]


# print unified_df.loc[(unified_df["category_id"]==category_id) & (unified_df["device_code"]==device_code) & (unified_df["country_code"]==country_code) & (unified_df["app_price_type_id"]==app_price_type_id) & (unified_df["purchase_type_id"]==purchase_type_id) ]


# print legacy_unified_df.loc[(legacy_unified_df["category_id"]==category_id) & (legacy_unified_df["device_code"]==device_code) & (legacy_unified_df["country_code"]==country_code) & (legacy_unified_df["app_price_type_id"]==app_price_type_id) & (legacy_unified_df["purchase_type_id"]==purchase_type_id) ]


print "(1000, datetime.date(2020, 1, 10), 14, 1, 65, 2, 12, 188L)"
# print legacy_unified_df
# print raw_df
# print diff_df.country_code.unique()
# print diff_df.category_id.unique()
# print diff_df.device_code.unique()



In [0]:


print raw_df

In [0]:
%%sh
# SELECT * FROM store.store_market_size_fact_v1  WHERE category_id=400007 AND app_price_type_id=1 AND purchase_type_id=12 AND country_code='CN' AND device_code='android-all' AND date = '2020-01-09'

PGPASSWORD='dNzWtSV3pKTx' psql -h 10.2.10.132 -U citus_bdp_usage_qa -d aa_citus_db -p 5432 << EOF 
SELECT count(*) FROM store.store_market_size_fact_v1  WHERE date = '2020-01-30'

EOF


In [0]:

# tom_df = tom_df.drop(["date"], axis=1)
# unified_df = unified_df.drop(["_identifier"], axis=1)
# legacy_unified_df = legacy_unified_df.drop(["_identifier"], axis=1)
pd.set_option('expand_frame_repr', False)

# print raw_df.dtypes
# print unified_df.dtypes
# print legacy_unified_df.dtypes


# print raw_df
# print unified_df
# print legacy_unified_df

df1= raw_df
df2= unified_df
# df2= legacy_unified_df

diff_df = df1.merge(df2,indicator = True, how='left', on=["app_price_type_id",  "purchase_type_id",  "category_id",  "device_code", "country_code", "est_market_size_download", "est_market_size_revenue" ])#.loc[lambda x : x['_merge']!='both']
# print df2[ ~df2.isin(df1)].dropna()

diff_df = diff_df.loc[ diff_df["_merge"]!="both"]

print diff_df
print diff_df.country_code.unique()
print diff_df.category_id.unique()
print diff_df.device_code.unique()

In [0]:
%%sh


aws s3 ls s3://b2c-prod-dca-store-estimates/store_est/v_final/DAY/2020-01-10/ios/market_size/143443/
aws s3 ls s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.market-size.v1/fact/granularity=daily/date=2020-01-10/


In [0]:
%%sh
# aws s3 ls s3://b2c-prod-dca-store-estimates/store_est/v_final/DAY/2020-02-01/ios/market_size/143441/
# aws s3 ls s3://b2c-prod-dca-store-estimates/store_est/v_final/DAY/2020-02-08/android/market_size/3/
# aws s3 ls s3://b2c-prod-dca-store-estimates/store_estv2/MARKET_SIZE_ESTIMATES_FINAL/version=2.0.0/range_type=DAY/date=2020-01-13/platform=android/
aws s3 ls s3://b2c-prod-dca-store-estimates/store_estv2/MARKET_SIZE_ESTIMATES_FINAL/version=2.0.0/range_type=DAY/date=2020-02-15/

aws s3 ls s3://b2c-prod-data-pipeline-raw-store-paid/
echo '***************************************************************************************'
# aws s3 ls s3://b2c-prod-dca-store-estimates/store_est/v_final/DAY/2020-02-01/android/market_size/1/
# aws s3 ls s3://b2c-prod-data-pipeline-unified-store-paid/unified/app-tech.store.market-size.v1/fact/granularity=daily/
# aws s3 ls s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.market-size.v1/fact/granularity=daily/


In [0]:


"""
+--------+----------+-----------+-----------+---------+----------+-------------+-----------+--------+
|store_id|      date|platform_id|     device|data_type|price_type|purchase_type|category_id|estimate|
+--------+----------+-----------+-----------+---------+----------+-------------+-----------+--------+
|      10|2020-01-10|          0|google-play|downloads|         1|           10|          1|11981138|
+--------+----------+-----------+-----------+---------+----------+-------------+-----------+--------+
    """
schema = StructType([
    StructField("store_id", IntegerType(), False),
    StructField("date", StringType(), False),
    StructField("platform_id", IntegerType(), False),
    StructField("device", StringType(), False),
    StructField("data_type", StringType(), False),
    StructField("price_type", IntegerType(), False),
    StructField("purchase_type", IntegerType(), False),
    StructField("category_id", IntegerType(), False),
    StructField("estimate", LongType(), False)
])
s3_path = "s3://b2c-prod-dca-store-estimates/store_est/v_final/DAY/2020-01-09/android/market_size/3/"
debug_raw_df = spark.read.csv(s3_path, schema=schema, sep='\t').toPandas()

print debug_raw_df.loc[ (debug_raw_df["category_id"]==44)]

print  debug_raw_df

In [0]:

from aadatapipelinecore.core.fs import Conf
from aadatapipelinecore.core.fs.device import S3Bucket, specified_bucket

from applications.db_check_v1.common.constants import COUNTRY_CODE_MAPPING_BY_MARKET_CODE as COUNTRY_CODE_MAPPING, \
    CATEGORY_ID_MAPPING_BY_MARLKET_AND_DEVICE_CODE as CATEGORY_ID_MAPPING


conf = Conf(
    bucket_name="b2c-prod-dca-store-estimates",
    bucket_class=S3Bucket
)
bucket = specified_bucket(conf)

date_list = ["2020-01-10", "2019-01-10", "2018-01-10"]

for store in ["ios", "android"]:
    for date in date_list:
        path = "store_est/v_final/DAY/{date}/{store}/market_size/".format(date=date, store=store)
        print "*" * 100
        print path
        for filepath in bucket.list(path, depth_is_1=True):
            # print filepath
            filename = filepath.replace(path, '').replace("/", '')
            country_id_list = COUNTRY_CODE_MAPPING["apple-store"].keys() if store=="ios" else COUNTRY_CODE_MAPPING["google-play"].keys()
            country_id_list = [str(country) for country in country_id_list]
            if filename not in country_id_list:
                print "{filename} should not in {path}".format(filename=filename, path=path)
                
    
 




In [0]:


spark.read.parquet("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.market-size.v1/fact/granularity=daily/date=2020-01-10/").printSchema()

spark.read.parquet("s3://b2c-prod-data-pipeline-unified-store-paid/unified/app-tech.store.market-size.v1/fact/granularity=daily/date=2020-01-10/").printSchema()

In [0]:


       app_price_type_id  category_id country_code  device_code  est_market_size_download  est_market_size_revenue  purchase_type_id
56749                  2       400070           ZA  android-all                     191.0                      NaN                12

In [0]:
%%sh

%%sh

PGPASSWORD='2mHdFW6%#REu' psql -h internal-aa-prod-plproxy-internal-4-329644124.us-east-1.elb.amazonaws.com -U app_bdp_usage_qa -d dailyest -p 7432 << EOF 
select device_id, date, store_id, kpi, category_id, app_type_id, purchase_type_id, estimate from plproxy.execute_select_nestloop(\$proxy\$ 
    select device_id, date, store_id, kpi, category_id, app_type_id, purchase_type_id, estimate
    from ms.market_size_daily_estimate_1000 
    where 
        date = '2020-01-10' and 
        purchase_type_id=12 and 
        app_type_id=2 and 
        store_id=14 and 
        category_id in (65) 
\$proxy\$) tbl (device_id SMALLINT, date DATE, store_id INT, kpi SMALLINT, category_id INT, app_type_id SMALLINT, purchase_type_id SMALLINT, estimate BIGINT);
EOF



In [0]:
%%sh
aws s3 ls s3://b2c-prod-data-pipeline-qa/tom/top_publisher/

aws s3 cp s3://b2c-prod-data-pipeline-qa/tom/top_publisher/regression1.txt /tmp/regression1.txt
aws s3 cp s3://b2c-prod-data-pipeline-qa/tom/top_publisher/regression2.txt /tmp/regression2.txt
aws s3 cp s3://b2c-prod-data-pipeline-qa/tom/top_publisher/regression3.txt /tmp/regression3.txt
aws s3 cp s3://b2c-prod-data-pipeline-qa/tom/top_publisher/regression4.txt /tmp/regression4.txt
aws s3 cp s3://b2c-prod-data-pipeline-qa/tom/top_publisher/regression5.txt /tmp/regression5.txt

cat /tmp/regression*.txt


In [0]:
%%sh
